Credits :

- Code pour un DC GAN sur MNIST avec Keras : https://github.com/kyloon/dcgan

# Sommaire

>[Sommaire](#scrollTo=92_l_n2EFOM6)

>[Imports et paramètres généraux](#scrollTo=4ZFzLCGtFxwB)

>[Chargement du Dataset](#scrollTo=3qDNgzbyF90r)

>>[Chargement des données](#scrollTo=w_YpePZHRxzm)

>>[Génération des variables de training](#scrollTo=JE_nu13MR08g)

>[Définition des modèles à entraîner](#scrollTo=dTd2vuuRGCgf)

>>[Générateur](#scrollTo=5ZBYXSdvGIV5)

>>[Discriminateur](#scrollTo=TrLP7R8iGKtf)

>>[GAN](#scrollTo=KlY4z0ICGSf7)

>[Fonctions d'affichages et d'enregistrements](#scrollTo=jZNJi7d7Gfa1)

>[Entraînement](#scrollTo=PLSQNDgHGjC7)

>[Génération](#scrollTo=CS1gscaOLzLU)

>>[Upload models](#scrollTo=gu1mm4S0OO2T)

>>[Generate](#scrollTo=1OMaw6J2OSnM)

>[Téléchargements](#scrollTo=Wv1YojWWGkts)



# Imports et paramètres généraux

Imports des librairies principales

In [0]:
from __future__ import print_function

import os
import numpy as np
from PIL import Image
import math
from keras.models import Sequential
from keras.layers import Dense, Reshape
from keras.layers.core import Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import SGD

Réglage des constantes

# Chargement du Dataset

## Chargement des données

In [0]:
from keras.datasets import mnist

## Génération des variables de training

In [0]:
def loadData() :
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])
    return (X_train, y_train), (X_test, y_test)

In [0]:
(X_train, Y_train), (X_test, Y_test) = loadData()

# Définition des modèles à entraîner

## Générateur

In [0]:
def generatorModel():
    model = Sequential()
    model.add(Dense(Dense(units=1024, input_dim=100))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((128,7,7), input_shape=(128*7*7,)))
    model.add(UpSampling2D(data_format="channels_first", size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding="same", data_format="channels_first"))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(data_format="channels_first", size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding="same", data_format="channels_first"))
    model.add(Activation('tanh'))
    return model

## Discriminateur

In [0]:
def discriminatorModel():
    model = Sequential()
    model.add(Conv2D(64, (5, 5), padding="same", data_format="channels_first", input_shape=(1, 28, 28))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5), padding="same", data_format="channels_first"))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

## GAN

 comme assemblage du Générateur et du Discriminateur

In [0]:
def ganModel(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

# Fonctions d'affichages et d'enregistrements

In [0]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]),
                      dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[0, :, :]
    return image

# Entraînement

In [0]:
def train(generator, discriminator, gan, X, Y, epochs=1, batchSize=128) :
  
  
    batchCount = int(X_train.shape[0]/batchSize)
  
    # Affichage des paramètres de train
    print('Epochs:', epochs)
    print('Batch size:', batchSize)
    print('Batches per epoch:', batchCount)
    
  
    # Liste des Coûts du Discriminateur (un par epoch)
    dLosses = []

    # Liste des Coûts du Discriminateur (un par epoch)
    gLosses = []
    
    
    # ENTRAINEMENT
    
    noise = np.zeros((batchSize, 100))
    
    for epoch in range(epochs):
      
        # Affichage du numéro de l'epoch
        print('-'*15, 'Epoch %d' % epoch, '-'*15)
        
        for index in range(batchCount):
          
            # On génère des données random de la bonne taille pour générer <batchSize> fausses images
            for i in range(batchSize):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            generated_images = generator.predict(noise, verbose=0)
                
            # On récupère un batch contenant des vraies images
            image_batch = X_train[index*batchSize:(index+1)*batchSize]

            # On construit un batch contenant vraies et fausses images
            X = np.concatenate((image_batch, generated_images))
            
            # On construit le Y cible correspondant
            y = [0.9] * batchSize + [0.1] * batchSize
          
            # Entrainement du discriminateur
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y)
            #print("Batch %d d_loss : %f" % (index+1, d_loss))
            
            # Entrainement du générateur via l'entrainement du GAN, le discriminateur étant fixé
            for i in range(batchSize):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, [1] * batchSize)
            
        generator.save('generator_'+str(epoch)+".hdf5")
        discriminator.save('discriminator_'+str(epoch)+".hdf5")
        
        image = combine_images(generated_images)
        image = image*127.5+127.5
        Image.fromarray(image.astype(np.uint8)).save("epoch_"+str(epoch)+".png")
                
    return dLosses,gLosses

In [0]:
generator = generatorModel()
generator.compile(loss='binary_crossentropy', optimizer="SGD")

discriminator = discriminatorModel()
discriminator.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.0005, momentum=0.9, nesterov=True))

gan = ganModel(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.0005, momentum=0.9, nesterov=True))

BATCH_SIZE = 128
NUM_EPOCHS = 100

dLosses,gLosses = train(generator, discriminator, gan, X_train, Y_train, epochs=NUM_EPOCHS, batchSize=BATCH_SIZE)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, input_dim=100)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `UpSampling2D` call to the Keras 2 API: `UpSampling2D(data_format="channels_first", size=(2, 2))`
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same", data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `UpSampling2D` call to the Keras 2 API: `UpSampling2D(data_format="channels_first", size=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Update you

Epochs: 100
Batch size: 128
Batches per epoch: 468
--------------- Epoch 0 ---------------
--------------- Epoch 1 ---------------


# Génération

## Upload models

In [0]:
generator = keras.models.load_model("generator_150.hdf5")

## Generate


In [0]:
def generate(batch_size, pretty=False):
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('generator_weights')
    if pretty:
        discriminator = discriminator_model()
        discriminator.compile(loss='binary_crossentropy', optimizer="SGD")
        discriminator.load_weights('discriminator_weights')
        noise = np.zeros((batch_size*20, 100))
        for i in range(batch_size*20):
            noise[i, :] = np.random.uniform(-1, 1, 100)
        generated_images = generator.predict(noise, verbose=1)
        d_pret = discriminator.predict(generated_images, verbose=1)
        index = np.arange(0, batch_size*20)
        index.resize((batch_size*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        pretty_images = np.zeros((batch_size, 1) +
                               (generated_images.shape[2:]), dtype=np.float32)
        for i in range(int(batch_size)):
            idx = int(pre_with_index[i][1])
            pretty_images[i, 0, :, :] = generated_images[idx, 0, :, :]
        image = combine_images(pretty_images)
    else:
        noise = np.zeros((batch_size, 100))
        for i in range(batch_size):
            noise[i, :] = np.random.uniform(-1, 1, 100)
        generated_images = generator.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save("generated_image.png")

# Téléchargements

In [0]:
from google.colab import files

files.download('gan_generated_image_epoch_%d.png' % e)